In [2]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

In [32]:
class Geant4Project():
    """ This class contains all the code I reuse as functions in DAML project 3 to make my Jupyter notebook cleaner.
    
    :param layer_total_file: .csv file of layer total data. 
    :type layer_total_file: str
    :param energy_file: .csv file of the energy measurements from the calorimeter.
    :type energy_file: str
    :param t1_file: .csv file of the inner tracker hits data.
    :type t1_file: str
    :param t2_file: .csv file of the outer tracker hits data.
    :type dates: str
    :param set_momentum: Define what the particle momentum was set at in the simulation.
    :type set_momentum: float
    :param scale: Define what the scale of the momentum is: 'MeV' or 'GeV'.
    :type scale: str
    :param edit: In my first dataset, my code didn't properly record the momentum in the energy data, so edit=True re-writes
                 that column with the correct values.
    :type edit: bool, optional
    """
    
    def __init__(self, layer_total_file, energy_file, t1_file, t2_file, set_momentum, scale, edit=False):
        """ Imports data and compiles it into dataframes with correct labels. Also sets the momentum scale.
        """
        self.layer_total = pd.read_csv(layer_total_file, comment='#')
        self.tracker1_hits = pd.read_csv(t1_file, comment='#', names=['Event', 'Phi', 'Theta'])
        self.tracker2_hits = pd.read_csv(t2_file, comment='#', names=['Event', 'Phi', 'Theta'])
        
        if edit is False:
            self.energy = pd.read_csv(energy_file, comment='#', names=['Momentum', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5'])
        elif edit is True:
            energy = pd.read_csv(energy_file, comment='#', names=['Momentum', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5'])
            energy['Momentum'] = energy['Momentum'] + set_momentum
            self.energy = energy
        
        self.actual_momentum = set_momentum
        
        if scale is 'GeV':
            self.multiplier = 10
        elif scale is 'MeV':
            self.multiplier = 1000
    
    def find_tracks(self, threshold=0.001):
        """ Creates a new dataframe with each row representing a reconstructed track based on information from the two tracker
        hits. The threshold determines how close the theta values of the events in trackers 1 and 2 must be for the hits to be
        the result of the same particle track.
        
        :param threshold: Value to determine how similar theta values must be to declare the set a pair.
        :type threshold: float, optional
        """
        # initializing empty lists for things I want to record
        event_ids = []
        t1_indices = []
        t2_indices = []
        t1_thetas = []
        t2_thetas = []
        t1_phis = []
        t2_phis = []

        events = np.arange(len(self.energy))

        # looping through event IDs, recording all of the information of
        # reconstructed particles that meet the requirement of having a delta theta
        # value less than 0.0001. I record the event ID, the indices of the particle
        # in both tracker datasets, and the theta angles of the particle in both 
        # datasets
        for event in events:
            # decreasing the datasets to subsets by event ID reduces loop time significantly
            subset1 = self.tracker1_hits[self.tracker1_hits['Event'] == event]
            subset2 = self.tracker2_hits[self.tracker2_hits['Event'] == event]

            for ind1 in subset1.index:
                for ind2 in subset2.index:
                    if abs(subset1['Theta'][ind1] - subset2['Theta'][ind2]) < threshold:
                        event_ids.append(event)
                        t1_indices.append(ind1)
                        t2_indices.append(ind2)
                        t1_thetas.append(subset1['Theta'][ind1])
                        t2_thetas.append(subset2['Theta'][ind2])
                        t1_phis.append(subset1['Phi'][ind1])
                        t2_phis.append(subset2['Phi'][ind2])

            # compiling the reconstructed particle information in a dataframe
            pairs = pd.DataFrame({'EventID': event_ids, 
                                  't1_index': t1_indices,
                                  't2_index': t2_indices,
                                  't1_theta': t1_thetas,
                                  't2_theta': t2_thetas,
                                  't1_phi': t1_phis,
                                  't2_phi': t2_phis
                                 })
            self.pairs = pairs
            
    def calc_sagitta(self, phi1, phi2, r1):
        """ Calculates the sagitta value to be used in momentum calculation.

        :param phi1: Individual phi value from tracker 1 data.
        :type phi1: float
        :param phi2: Individual phi value from tracker 2 data.
        :type phi2: float
        :param r1: Radius of the inner tracker [meters].
        :type r1: float
        """
        delta_phi = abs(phi1 - phi2)
        sagitta = np.sin(delta_phi)*r1
        return sagitta

    def calc_momentum(self, r2, sagitta, B, theta):
        """ Calculates a single momentum value.

        :param r2: Radius of the outer tracker [meters].
        :type r2: float
        :param sagitta: Sagitta value (can be calculated with calc_sagitta).
        :type phi2: float
        :param B: Magnitude of the magnetic field [Tesla].
        :type B: float
        :param theta: Theta value of the particle.
        :type theta: floa
        """
        R = (r2**2) / (8*sagitta)
        p_T = 0.3*B*R
        p = p_T / np.sin(theta)
        return p
        
    def find_momentum(self, R1=4, R2=8, B=0.2):
        """ Calculates the momenta for all pairs in self.pairs.

        :param R1: Radius of the inner detector [meters].
        :type R1: float, optional
        :param R2: Radius of the outer detector [meters].
        :type R2: float, optional
        :param B: Magnitude of the magnetic field [Tesla].
        :type B: float, optional
        """
        # initializing an empty list to record calculated momenta
        calculated_momenta = []

        # looping through every reconstructed particle in the pairs dataframe, using the functions
        # in the previous cell to calculate total momentum. I chose just to use the theta value
        # in the tracker1 dataset, since it is essentially equal to the theta in tracker2 data
        for item in self.pairs.index:
            index_1 = self.pairs['t1_index'][item]
            index_2 = self.pairs['t2_index'][item]

            s = self.calc_sagitta(self.tracker1_hits['Phi'][index_1], self.tracker2_hits['Phi'][index_2], R1)
            p = self.calc_momentum(R2, s, B, self.pairs['t1_theta'][item])*self.multiplier

            calculated_momenta.append(p)

        # concatenating the momentum data and the pairs dataframe
        momenta_df = pd.DataFrame({'momentum': calculated_momenta})
        self.pairs_updated = pd.concat([self.pairs, momenta_df], axis=1)

    def plot_momentum(self, title='Momentum Distribution', xlabel='Momentum [GeV]', bin_num=15):
        """ Plots the momentum distribution of the dataset.

        :param title: Plot title.
        :type title: str, optional
        :param xlabel: Label of the x-axis.
        :type xlabel: str, optional
        :param bin_num: Number of bins used in the histogram.
        :type bin_num: float, optional
        """
        plt.figure()
        plt.hist(self.pairs_updated['momentum'], bins=bin_num)
        plt.xlabel(xlabel)
        plt.ylabel('Counts')
        plt.title(title)
        plt.show()

    def classify_particles(self, threshold=0.001):
        """ Separates particles into photons and electrons. Generates a 'photon' dataframe and an 'electron' dataframe.

        :param threshold: Determines how strictly the phi values in the two tracker datasets must match.
        :type threshold: float, optional
        """
        # using a similar method to how I determined whether particles were 'pairs' or not
        photon_indices = []
        electron_indices = []

        for index, row in self.pairs.iterrows():
            phi_diff = abs(row['t1_phi'] - row['t2_phi'])

            if phi_diff < threshold:
                photon_indices.append(index)
            else:
                electron_indices.append(index)


        # saving these values in separate dataframes:
        photons = self.pairs.loc[photon_indices]
        electrons = self.pairs.loc[electron_indices]

        self.photons = photons
        self.electrons = electrons

    def calc_resolution(self):
        """ Calculates the momentum resolution for all momenta recorded in pairs_updated['momenta'].
        """
        res_vals = []
        column = self.pairs_updated['momentum']
        actual_val = self.actual_momentum

        for value in column:
            diff = (value - actual_val)/actual_val
            res_vals.append(diff)
        return res_vals

    def plot_resolution(self, title='Momentum Resolution Distribution', bin_num=20):
        """ Plots the momentum resolution histogram.

        :param title: Plot title.
        :type title: str, optional
        :param bin_num: Number of bins used in the histogram.
        :type bin_num: float, optional
        """
        res_vals = self.calc_resolution()
        plt.figure()
        plt.hist(res_vals, bins=bin_num)
        plt.xlabel('(Calculated Momentum - True Momentum)/True Momentum')
        plt.ylabel('Number of Reconstructed Particles')
        plt.title(title)
        plt.show()